In [14]:
# Databricks notebook source
import pandas as pd
import subprocess
from datetime import datetime

# COMMAND ----------
# DBTITLE 1,Schema Check Defined
def perform_schema_check(df, expected_columns):
    # Checking the DataFrame has the expected columns
    missing_columns = set(expected_columns) - set(df.columns)
    new_line ='\n'

    if missing_columns:
        return f"Missing columns: {', '.join(missing_columns)}"

    # Checking if the column names match
    if set(expected_columns) != set(df.columns): 
        return f"""COLUMN NAMES DO NOT MATCH. <br> EXPECTED COLUMN NAMES ARE : {', '.join(expected_columns)} 
        <br>
        <br>
        THE ACTUAL NAMES WE HAVE ARE : {', '.join(df.columns)}"""

    return None

# COMMAND ----------
# DBTITLE 1,Data Quality Rules Defined

def region_null(row):
  if row["MARKET"] is None:
      return "error"
  else:
      return "no issue"

def brand_null(row):
  if row["BRAND"] is None:
      return "error"
  else:
      return "no issue"

def sub_brand_null(row):
    if row["SUB_BRAND"] is None:
        return "error"
    else:
        return "no issue"

def category_null(row):
    if row["CATEGORY"] is None:
        return "error"
    else:
        return "no issue"

def Item_id_null(row):
    if row["ITEM_ID"] is None:
        return "error"
    else:
        return "no issue"

def week_null(row):
    if row["WEEK"] is None:
        return "error"
    else:
        return "no issue"

def value_sales_is_null(row):
    if pd.isna(row["SUM_SLS_VAL_LCLCCY_AMT"]):
        return "error"
    else:
        return "no issue"
    
def volume_sales_is_null(row):
    if pd.isna(row["SUM_SLS_VOL"]):
        return "error"
    else:
        return "no issue"

# COMMAND ----------

# DBTITLE 1,Duplicate Check Defined
def identify_duplicates(dataframe):
    result_data = []

    unique_rows = {}

    for index, row in dataframe.iterrows():
        row_tuple = tuple([val if pd.notna(val) else 'nan' for val in row])
        
        if row_tuple in unique_rows:
            unique_rows[row_tuple].append(index)
        else:
            unique_rows[row_tuple] = [index]
    

    for row_tuple, indices in unique_rows.items():
        if len(indices) > 1:
            result_data.append(([index + 2 for index in indices ], 'duplicate'))
        else:
            result_data.append((indices, 'unique'))

    result_df = pd.DataFrame(result_data, columns=['duplicate_indices', 'rule_result'])
    result_df['rule_name'] = 'duplicate'
    result_df['duplicate_indices'] = result_df['duplicate_indices'].apply(lambda indices: '|'.join(map(str, indices)))
    result_df = pd.concat([dataframe, result_df], axis=1)
    return result_df

# COMMAND ----------
# DBTITLE 1,Main Method
def main():
  # path = r"/Workspace/Users/u419479@wns.com/Input/"
  getPath = r"D:/Project/TCS project/Node/src/data/getFile/"
  setPath = r"D:/Project/TCS project/Node/src/data/setFile/"
  raw_df = pd.read_excel(getPath+"poc_data_raw.xlsx")
  print("\n Dataframe Read Completed")
  print("\n Initiating Schema Check")
  expected_columns = ['SOURCE','CHANNEL','GRANULARITY','HIERARCHY','WEEK','MARKET','GEO_LEVEL','GEO_UNIT','BRAND','SUB_BRAND','MANUFACTURER','CATEGORY','SUB_CATEGORY','SEGMENT','SUB_SEGMENT','ITEM_ID','MULTIPACK_QTY_VAL','PACKAGE_TYPE','SERVING_TYPE','DSTRBTN_CHCKOUT_NUMRC','DSTRBTN_CHCKOUT_WGHTD','DSTRBTN_CHLLD_WGHTD','DSTRBTN_NUMRC',
  'DSTRBTN_OOS_NUMRC','DSTRBTN_OOS_WGHTD','DSTRBTN_WGHTD','DSTRBTN_WGHTD_ANY_PROMO','DSTRBTN_WGHTD_DSPLY','DSTRBTN_WTD_FEATR','DSTRBTN_WGHTD_FEATR_AND_DSPLY','SUM_STOR_CNT','SUM_SLS_UNITS','SUM_SLS_VAL_LCLCCY_AMT','SUM_SLS_VOL', 'SUM_TOT_DSTRBTN_PTS','DISTR_WTD_VOL']
  var_out = perform_schema_check(raw_df, expected_columns)
  if var_out == None:
    print("\n Schema checks have passed")
  else:
    print("\n Schema checked have failed")
    print("\n"+var_out)
  print("\n Define the Data Quality Rules")
  data_quality_rules = {"region_null" : region_null, "brand_null" : brand_null, "sub_brand_null" : sub_brand_null, "category_null" : category_null,"Item_id_null":Item_id_null,"week_null": week_null,"value_sales_is_null": value_sales_is_null, "volume_sales_is_null": volume_sales_is_null}
  print("\n Setting up the row id's")
  raw_df = raw_df.reset_index(drop=True)
  raw_df['RID'] = raw_df.index+1
  column_name = ['DSTRBTN_CHCKOUT_NUMRC','DSTRBTN_CHCKOUT_WGHTD','DSTRBTN_CHLLD_WGHTD','DSTRBTN_NUMRC','DSTRBTN_OOS_NUMRC','DSTRBTN_OOS_WGHTD','DSTRBTN_WGHTD','DSTRBTN_WGHTD_ANY_PROMO','DSTRBTN_WGHTD_DSPLY','DSTRBTN_WTD_FEATR','DSTRBTN_WGHTD_FEATR_AND_DSPLY','SUM_STOR_CNT','SUM_SLS_UNITS','SUM_SLS_VAL_LCLCCY_AMT','SUM_SLS_VOL','SUM_TOT_DSTRBTN_PTS','DISTR_WTD_VOL']
  raw_df[column_name] = raw_df[column_name].apply(pd.to_numeric, errors='coerce', axis=1)
  date_columns = ['WEEK']
  print("\n Running the Data Quality Rules")
  for col in date_columns:
    raw_df[col] = pd.to_datetime(raw_df[col]).dt.date
  result_df_pre = raw_df.copy()
  # result_df_pre=pd.DataFrame()
  for rule_name, function in data_quality_rules.items():
    result_df_pre[rule_name] = raw_df.apply(function, axis=1)
  
  result_df = result_df_pre.melt(id_vars=raw_df.columns, var_name="rule_name", value_name="rule_result")
  print("\n Data Quality Rules Execution Completed")

# Dividing the results in to three parts based on the rule  results 
  expected_columns_final = expected_columns + ['rule_name','rule_result']
  warning_df = result_df[result_df['rule_result'] == 'warning']
  warning_df_final = warning_df[expected_columns_final]

  warning_df_final.to_json(setPath+'input_war.json', orient='records')
  print("\n Warning Data Json Created")
  no_issue_df = result_df[result_df['rule_result'] == 'no issue']
  
  no_issue_df.to_json(setPath+'input_clean.json')
  print("\n No Issues Data Json Created")
  no_issue_df.to_excel(setPath+'input_clean.xlsx')
  
  error_df = result_df[result_df['rule_result'] == 'error']
  error_df_final= error_df[expected_columns_final]
  error_df_final.to_json(setPath+'input_error.json', orient='records')
  print("\n Errors Data Json Created")
  column_to_drop = 'RID'
  pandas_df_new = raw_df.drop(column_to_drop,axis = 1)
  pandas_df_new = pandas_df_new.reset_index(drop=True)
  duplicate_df = identify_duplicates(pandas_df_new)
  duplicate_df = duplicate_df[duplicate_df['rule_result'] == 'duplicate']
  duplicate_df_final = duplicate_df[expected_columns_final]
  duplicate_df_final.to_json(setPath+'input_dup.json', orient='records')
  print("\n Duplicate Data Check Completed and Json Generated")
  nbr_rows_in_media = len(raw_df)
  nbr_rows_with_error = len(error_df_final)
  nbr_rows_with_warning = len(warning_df_final)
  nbr_rows_with_duplicate = len(duplicate_df_final)
  overview_dict = {}
  overview_dict['total_rows'] = [nbr_rows_in_media]
  overview_dict['total_failed_dq_check'] = [nbr_rows_with_error]
  overview_dict['total_warning_dq_check'] = [nbr_rows_with_warning]
  overview_dict['total_duplicate_dq_check'] = [nbr_rows_with_duplicate]
  overview_df = pd.DataFrame.from_dict(overview_dict).T
  overview_df.columns = ['Overview']
  overview_df.to_json(setPath+'input_dq_summ.json')
  print("\n DQ Summary Report Generated")

if __name__==main():
  main()



 Dataframe Read Completed

 Initiating Schema Check

 Schema checks have passed

 Define the Data Quality Rules

 Setting up the row id's

 Running the Data Quality Rules

 Data Quality Rules Execution Completed

 Warning Data Json Created

 No Issues Data Json Created

 Errors Data Json Created

 Duplicate Data Check Completed and Json Generated

 DQ Summary Report Generated


In [4]:
#!/usr/bin/env python
# coding: utf-8

# # Master Table creation & Loading

# In[4]:
import pandas as pd
import datetime as dt
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

# In[5]:
def fuzzy_check(df_source,df_reference):
    df_source['fuzzy_check_source']=df_source["BRAND"]+"-"+df_source["SUB_BRAND"]
    df_reference['fuzzy_check_reference']=df_reference["PROD_BRND"]+"-"+df_reference["PROD_SUB_BRND"]
    cat_list = list(set(list(df_source["CATEGORY"])))
    source_list_temp=list()
    final_brand_subbrand_list=list()
    for i in cat_list:
        df_source_v1 = df_source.loc[df_source['CATEGORY'] == i]
        df_reference_v1=df_reference.loc[df_reference['PROD_CTGY'] == i]
        source_list = list(df_source_v1['fuzzy_check_source'])
        ref_list=list(set(list(df_reference_v1['fuzzy_check_reference'])))
        for i in source_list:
            try:
                if i in ref_list:
                    source_list_temp.append(i)
                elif i not in ref_list:
                    for j in ref_list:
                        if fuzz.WRatio(i, j)>=95:
                            source_list_temp.append(j)
                        else:
                            pass
                
                    source_list_temp.append('NO MATCH')
            except Exception as e:
                print(e)
                        
    df_source['brand_check']=source_list_temp
    return df_source

# In[6]:
#Fetch Brand codes and final dataframe
def create_brand_code(df_source,df_reference):
    df_reference['fuzzy_check_reference']=df_reference["PROD_BRND"]+"-"+df_reference["PROD_SUB_BRND"]
    df_final = pd.merge(df_source,df_reference,left_on=['CATEGORY','brand_check'],\
                                  right_on=['PROD_CTGY','fuzzy_check_reference'])[:]
    df_final['Week_1']=df_final['WEEK'].dt.strftime('%d-%m-%Y')
    return df_final
    
# # Build Master Dim Table 
# #Step 1: Create a Date Dimension Table

# In[7]:
def create_date_table2(start='2000-01-01', end='2050-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df['Date_ref'] = df['Date'].dt.strftime('%d-%m-%Y')
    df["Day"] = df['Date'].dt.day_name()
    df["Week"] = df.Date.dt.isocalendar().week
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    df.insert(0, 'date_id', range(1, 1 + len(df)))
    return df

# In[8]:
# Create Region Master
def create_cntry_master(df_input, path):
    country_list = list(set(list(df_input['MARKET'])))
    df_cntry= pd.DataFrame()
    df_cntry['cntry']=country_list
    df_cntry.insert(0,'cntry_id',range(1,1+len(df_cntry)))
    df_cntry.to_json(path+'countryMaster.json', orient='records')
    return df_cntry

# In[9]:
#brand Master
def create_brand_master(df_input, path):
    brand_list = list(set(list(df_input['BRAND'])))
    df_brand= pd.DataFrame()
    df_brand['brand']=brand_list
    df_brand.insert(0,'brand_id',range(1,1+len(df_brand)))
    df_brand.to_json(path+'brandMaster.json', orient='records')
    return df_brand

# In[10]:
#create Sub brand Master
def create_sub_brand_master(df_input,df_brand, path):
    sub_brand_list = list(df_input['SUB_BRAND'])
    brand_list= list(df_input['BRAND'])
    brand_code_list = list(df_input['BRAND_CODE'])
    df_sub_brand = pd.DataFrame()
    df_sub_brand['sub_brand']=sub_brand_list
    df_sub_brand['brand']=brand_list
    df_sub_brand['brand_code']=brand_code_list
    df_sub_brand_int = pd.merge(df_sub_brand,df_brand,left_on="brand",right_on='brand')[['sub_brand','brand_id',\
                                                                                         'brand_code']]
    df_sub_brand_int.insert(0,'sub_brand_id',range(1,1+len(df_sub_brand_int)))
    df_sub_brand_int.to_json(path+'subBrandMaster.json', orient='records')
    return df_sub_brand_int
    
# In[11]:
#category Master
def create_cat_master(df_input, path):
    cat_list = list(set(list(df_input['CATEGORY'])))
    df_cat= pd.DataFrame()
    df_cat['category']=cat_list
    df_cat.insert(0,'cat_id',range(1,1+len(df_cat)))
    df_cat.to_json(path+'categoryMaster.json', orient='records')
    return df_cat

# In[12]:
#Sub Category Master
def create_sub_cat_master(df_input,df_cat,path):
    sub_cat_list = list(df_input['SUB_CATEGORY'])
    cat_list= list(df_input['CATEGORY'])
    df_sub_cat_master = pd.DataFrame()
    df_sub_cat_master['sub_category']=sub_cat_list
    df_sub_cat_master['category']=cat_list
    df_sub_cat_master_int = pd.merge(df_sub_cat_master,df_cat,left_on ="category",right_on="category")[['sub_category',\
                                                                                                        'cat_id']]
    df_sub_cat_master_int=df_sub_cat_master_int.drop_duplicates()
    df_sub_cat_master_int.insert(0,'sub_cat_id',range(1,1+len(df_sub_cat_master_int)))
    df_sub_cat_master.to_json(path+'subCategoryMaster.json', orient='records')
    return df_sub_cat_master_int

# In[13]:
#create Segment Master
def create_seg_master(df_input,path):
    seg_master_list = list(set(list(df_input['SEGMENT'])))
    df_segment_mstr = pd.DataFrame()
    df_segment_mstr['segment']=seg_master_list
    df_segment_mstr.insert(0,'seg_id',range(1,1+len(df_segment_mstr)))
    df_segment_mstr.to_json(path+'segmentMaster.json', orient='records')
    return df_segment_mstr
    
# In[14]:
#create Sub Segment Master
def create_sub_seg_mstr(df_input,df_segment_mstr,path):
    seg_list = list(df_input['SEGMENT'])
    sub_seg_list = list(df_input['SUB_SEGMENT'])
    df_sub_seg = pd.DataFrame()
    df_sub_seg['segment']=seg_list
    df_sub_seg['sub_segment']=sub_seg_list
    df_sub_segment_int = pd.merge(df_sub_seg,df_segment_mstr,left_on="segment",right_on="segment")[['sub_segment',\
                                                                                                    'seg_id']]
    df_sub_segment_int.insert(0,'sub_seg_id',range(1,1+len(df_sub_segment_int)))
    df_sub_segment_int.to_json(path+'subSegmentMaster.json', orient='records')
    return df_sub_segment_int

# # Build Fact Table
# In[15]:
#sub brand Master
def create_fact(df_input,df_sub_segment,df_sub_cat,df_sub_brand,df_cntry,df_date,path):
    df_fact_int=pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(df_input,df_sub_segment,left_on ="SUB_SEGMENT",right_on="sub_segment"),\
                         df_sub_cat,left_on="SUB_CATEGORY",right_on="sub_category"),df_sub_brand,left_on="SUB_BRAND",\
                         right_on="sub_brand"),df_cntry,left_on="MARKET",right_on="cntry"),df_date,left_on="Week_1",\
                         right_on='Date_ref')[['ITEM_ID','SUM_STOR_CNT','SUM_SLS_UNITS','SUM_SLS_VAL_LCLCCY_AMT',\
                                               'SUM_SLS_VOL','SUM_TOT_DSTRBTN_PTS','DISTR_WTD_VOL','sub_seg_id','seg_id',\
                                              'cat_id','sub_cat_id','sub_brand_id','brand_id','brand_code','cntry_id',\
                                              'date_id']]
    df_fact_int.insert(0,'fact_id',range(1,1+len(df_fact_int)))
    df_fact_int.to_json(path+'factMaster.json', orient='records')
    return df_fact_int

# In[16]:
def main():
     # path = r"/Workspace/Users/u419479@wns.com/Input/"
    getPath = r"D:/Project/TCS project/Node/src/data/getFile/"
    setPath = r"D:/Project/TCS project/Node/src/data/setFile/"
    masterPath = r"D:/Project/TCS project/Node/src/data/masterFile/"
    df_source_ini = pd.read_excel(setPath+'input_clean.xlsx')
    df_source_int=df_source_ini[['SOURCE','CHANNEL','GRANULARITY','HIERARCHY','WEEK','MARKET','GEO_LEVEL','GEO_UNIT','BRAND','SUB_BRAND','MANUFACTURER','CATEGORY','SUB_CATEGORY','SEGMENT','SUB_SEGMENT','ITEM_ID','MULTIPACK_QTY_VAL','PACKAGE_TYPE','SERVING_TYPE','DSTRBTN_CHCKOUT_NUMRC','DSTRBTN_CHCKOUT_WGHTD','DSTRBTN_CHLLD_WGHTD','DSTRBTN_NUMRC',
  'DSTRBTN_OOS_NUMRC','DSTRBTN_OOS_WGHTD','DSTRBTN_WGHTD','DSTRBTN_WGHTD_ANY_PROMO','DSTRBTN_WGHTD_DSPLY','DSTRBTN_WTD_FEATR','DSTRBTN_WGHTD_FEATR_AND_DSPLY','SUM_STOR_CNT','SUM_SLS_UNITS','SUM_SLS_VAL_LCLCCY_AMT','SUM_SLS_VOL', 'SUM_TOT_DSTRBTN_PTS','DISTR_WTD_VOL']]
    df_source=df_source_int.drop_duplicates()
    df_reference= pd.read_csv(getPath+'store_mapping_sample_fr.csv')
    print("\n Run Fuzzy check for brand & sub Brand codes")
    df_source = fuzzy_check(df_source,df_reference)
    print("\n----------------------------------------------------")
    print("\n Fuzzy Check for brand and Sub Brand Codes Completed")
    df_input= create_brand_code(df_source,df_reference)
    print("\n The source DataFrame has been created with Week and Product Brand code")
    print("\n----------------------------------------------------")
    print("\n Building the master Table:{0}".format("Date Master"))
    df_date = create_date_table2()
    print("\n Data Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the master Table:{0}".format("Region Master"))
    df_cntry = create_cntry_master(df_input,masterPath)
    print("\n Region Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the master Table:{0}".format("Brand Master"))
    df_brand = create_brand_master(df_input,masterPath)
    print("\n Brand Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Sub Brand Master"))
    df_sub_brand=create_sub_brand_master(df_input,df_brand,masterPath)
    print("\n Sub Brand Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Catgory Master"))
    df_cat=create_cat_master(df_input,masterPath)
    print("\n Category Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Sub Catgory Master"))
    df_sub_cat=create_sub_cat_master(df_input,df_cat,masterPath)
    print("\n Sub Category Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("segment Master"))
    df_segment_mstr=create_seg_master(df_input,masterPath)
    print("\n Segment Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("sub segment Master"))
    df_sub_segment = create_sub_seg_mstr(df_input,df_segment_mstr,masterPath)
    print("\n Sub Segment Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Fact"))
    df_fact= create_fact(df_input,df_sub_segment,df_sub_cat,df_sub_brand,df_cntry,df_date,masterPath)
    print("\n Fact Table Created")
    print("\n----------------------------------------------------")
    print("\n Process has completed")
    
if __name__=="__main__": 
    main()
    


 Run Fuzzy check for brand & sub Brand codes

----------------------------------------------------

 Fuzzy Check for brand and Sub Brand Codes Completed

 The source DataFrame has been created with Week and Product Brand code

----------------------------------------------------

 Building the master Table:Date Master

 Data Master Created

----------------------------------------------------

 Building the master Table:Region Master

 Region Master Created

----------------------------------------------------

 Building the master Table:Brand Master

 Brand Master Created

----------------------------------------------------

 Building the Master Table:Sub Brand Master

 Sub Brand Master Created

----------------------------------------------------

 Building the Master Table:Catgory Master

 Category Master Created

----------------------------------------------------

 Building the Master Table:Sub Catgory Master

 Sub Category Master Created

--------------------------------------

C:\Users\Ronak.pandya\AppData\Local\Temp\ipykernel_16532\679728314.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source['fuzzy_check_source']=df_source["BRAND"]+"-"+df_source["SUB_BRAND"]
C:\Users\Ronak.pandya\AppData\Local\Temp\ipykernel_16532\679728314.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source['brand_check']=source_list_temp



 Fact Table Created

----------------------------------------------------

 Process has completed
